In [1]:
data <- read.csv("HBSC2018OAed1.csv")
head(data)

,HBSC,seqno_int,cluster,countryno,region,id1,id2,id3,id4,weight,⋯,talkstepmo,famhelp,famsup,famtalk,famdec,MBMI,IRFAS,IRRELFAS_LMH,IOTF4,oweight_who
,<int>,<int>,<int>,<int>,<chr>,<int>,<int>,<dbl>,<int>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,2018,101054,100001,8000,AL,1,1,1,937,1,⋯,5,7,7,7,7,18.06617,8,2,2,0
2,2018,103215,100001,8000,AL,1,1,1,941,1,⋯,Inf,7,7,7,7,Inf,7,2,Inf,Inf
3,2018,103524,100001,8000,AL,1,1,1,934,1,⋯,5,7,7,7,7,14.87210,11,3,1,0
4,2018,101053,100002,8000,AL,1,1,2,936,1,⋯,Inf,7,7,7,7,20.00000,12,3,2,1
5,2018,100812,100002,8000,AL,1,1,2,947,1,⋯,Inf,7,7,7,7,21.41094,8,2,3,1
6,2018,100811,100002,8000,AL,1,1,2,940,1,⋯,Inf,7,7,7,7,19.28938,7,2,2,0


In [2]:
# Select specific columns by column names
selected_data <- subset(data, select = c("lifesat", "countryno", "fasholidays", "health", "age", "sex", "motherhome1", "fatherhome1", "IRRELFAS_LMH", "friendhelp", "schoolpressure"))
#colnames(selected_data)["IRRELFAS_LMH"] <- "famaffluence"
names(selected_data)[names(selected_data) == "IRRELFAS_LMH"] <- "famaffluence"
head(selected_data)

# Print the number of rows
num_rows <- nrow(selected_data)
print(num_rows)

,lifesat,countryno,fasholidays,health,age,sex,motherhome1,fatherhome1,famaffluence,friendhelp,schoolpressure
,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,9,8000,1,1,12.00000,1,1,1,2,7,1
2,10,8000,2,1,12.83333,1,1,1,2,5,2
3,9,8000,4,1,12.58333,2,1,1,3,7,3
4,9,8000,3,2,12.00000,1,1,1,3,7,2
5,10,8000,1,1,12.08333,1,1,1,2,1,4
6,9,8000,2,1,12.08333,2,1,1,2,7,3


[1] 240951


In [3]:
# remove all rows that contain a missing value which are stored as an inf large #
hbsc_data <- selected_data[!apply(selected_data > 840000, 1, any), ]

# create indicators and transformations
hbsc_data$fasholidays[hbsc_data$fasholidays == 1] <- 0
hbsc_data$fasholidays[hbsc_data$fasholidays == 2] <- 1
hbsc_data$fasholidays[hbsc_data$fasholidays == 3] <- 2
hbsc_data$fasholidays[hbsc_data$fasholidays == 4] <- 3
hbsc_data$lifesat <- hbsc_data$lifesat + 1
hbsc_data$llifesat <- log(hbsc_data$lifesat)
hbsc_data$fasholidays <- hbsc_data$fasholidays - 1
hbsc_data$female_indicator <- hbsc_data$sex - 1
hbsc_data$mhome_indicator <- hbsc_data$motherhome1 *-1 + 2
hbsc_data$fhome_indicator <- hbsc_data$fatherhome1 *-1 + 2

In [4]:
# One-hot encode
#encoded_data <- cbind(hbsc_data, model.matrix(~ factor(fasholidays) - 1, data = hbsc_data))
encoded_data <- cbind(hbsc_data, model.matrix(~ factor(health) - 1, data = hbsc_data))
encoded_data <- cbind(encoded_data, model.matrix(~ factor(famaffluence) - 1, data = encoded_data))
encoded_data <- cbind(encoded_data, model.matrix(~ factor(schoolpressure) - 1, data = encoded_data))

# Remove the original variable
#encoded_data <- encoded_data[, !(names(encoded_data) %in% c("fasholidays"))]
encoded_data <- encoded_data[, !(names(encoded_data) %in% c("health"))]
encoded_data <- encoded_data[, !(names(encoded_data) %in% c("famaffluence"))]
encoded_data <- encoded_data[, !(names(encoded_data) %in% c("schoolpressure"))]

# Rename the dummy variables
#colnames(encoded_data) <- gsub("factor\\(fasholidays\\)", "fasholidays", colnames(encoded_data))
colnames(encoded_data) <- gsub("factor\\(health\\)", "health", colnames(encoded_data))
colnames(encoded_data) <- gsub("factor\\(famaffluence\\)", "famaffluence", colnames(encoded_data))
colnames(encoded_data) <- gsub("factor\\(schoolpressure\\)", "schoolpressure", colnames(encoded_data))

# View the resulting data frame
head(encoded_data)

,lifesat,countryno,fasholidays,age,sex,motherhome1,fatherhome1,friendhelp,llifesat,female_indicator,⋯,health2,health3,health4,famaffluence1,famaffluence2,famaffluence3,schoolpressure1,schoolpressure2,schoolpressure3,schoolpressure4
,<dbl>,<int>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,10,8000,-1,12.00000,1,1,1,7,2.302585,0,⋯,0,0,0,0,1,0,1,0,0,0
2,11,8000,0,12.83333,1,1,1,5,2.397895,0,⋯,0,0,0,0,1,0,0,1,0,0
3,10,8000,2,12.58333,2,1,1,7,2.302585,1,⋯,0,0,0,0,0,1,0,0,1,0
4,10,8000,1,12.00000,1,1,1,7,2.302585,0,⋯,1,0,0,0,0,1,0,1,0,0
5,11,8000,-1,12.08333,1,1,1,1,2.397895,0,⋯,0,0,0,0,1,0,0,0,0,1
6,10,8000,0,12.08333,2,1,1,7,2.302585,1,⋯,0,0,0,0,1,0,0,0,1,0


In [5]:
model <- lm(llifesat ~ fasholidays+female_indicator+mhome_indicator+fhome_indicator+health1+health2+health4+famaffluence1+famaffluence3 + friendhelp + schoolpressure1 + schoolpressure3 + schoolpressure4, data = encoded_data)

# Print a summary of the regression results
summary(model)


Call:
lm(formula = llifesat ~ fasholidays + female_indicator + mhome_indicator + 
    fhome_indicator + health1 + health2 + health4 + famaffluence1 + 
    famaffluence3 + friendhelp + schoolpressure1 + schoolpressure3 + 
    schoolpressure4, data = encoded_data)

Residuals:
     Min       1Q   Median       3Q      Max 
-2.34750 -0.08429  0.04628  0.14656  0.95484 

Coefficients:
                   Estimate Std. Error t value Pr(>|t|)    
(Intercept)       1.8599223  0.0032861 565.989  < 2e-16 ***
fasholidays       0.0116298  0.0006062  19.183  < 2e-16 ***
female_indicator -0.0195782  0.0011695 -16.740  < 2e-16 ***
mhome_indicator   0.0445063  0.0024079  18.483  < 2e-16 ***
fhome_indicator   0.0422526  0.0013465  31.378  < 2e-16 ***
health1           0.2290488  0.0019431 117.878  < 2e-16 ***
health2           0.1412988  0.0018467  76.514  < 2e-16 ***
health4          -0.2778904  0.0049659 -55.960  < 2e-16 ***
famaffluence1    -0.0332048  0.0015860 -20.936  < 2e-16 ***
famaffluence3    